In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import datetime
import pandas as pd
import sys
import os
import math
sys.path.append('../..')
import scipy.optimize as opt

from models.optim.gaboh_choice_opt import SEIR

In [5]:
# intervention = [np.array([start days]), np.array([duration]), np.array([choice])]
def calculate_opt(intervention):
    R0 = 2.2 
    T_inf = 2.9
    T_trans = T_inf/R0
    T_inc = 5.2
    T_recov_mild = (14 - T_inf)
    T_hosp = 5
    T_recov_severe = (31.5 - T_inf)
    T_death = 32

    P_severe = 0.2
    P_fatal = 0.02
    P_mild = 1 - P_severe - P_fatal

    N = 7e6
    I0 = 1.0

    # new_R0 = 0.74
    # intervention_amount = R0/new_R0
    intervention_day = intervention[0]
    intervention_duration = intervention[1]
    intervention_choice = intervention[2]

    assert(len(intervention_day) == len(intervention_duration))
    assert(len(intervention_duration) == len(intervention_choice))
    
    params = [T_trans, T_inc, T_inf, T_recov_mild, T_hosp, T_recov_severe, T_death, 
              P_mild, P_severe, P_fatal, N, intervention_day, intervention_duration, intervention_choice]

    # S, E, I, R_mild, R_severe, R_severe_home, R_fatal, C, D
    state_init_values = [(N - I0)/N, 0, I0/N, 0, 0, 0, 0, 0, 0]
    
    solver = SEIR(params, state_init_values)
    sol = solver.solve_ode(time_step=1, total_no_of_days=400)
    states_int_array = (sol.y*N).astype('int')
    
    E = states_int_array[1]
    I = states_int_array[2]
    H = states_int_array[5]
    F = states_int_array[8]
    
    # S_coeficeint=1
    # E_coeficeint=0.825
    # I_coeficeint=0.75
    # R_mild_coeficeint=0.625
    # R_severe_coeficeint=0.5
    # R_severe_home_coeficeint=0.325
    # R_R_fatal_coeficeint=0.25
    # C_coeficeint=1
    # D_coeficeint=0
    
    S_coeficeint=1
    E_coeficeint=0
    I_coeficeint=0
    R_mild_coeficeint=0
    R_severe_coeficeint=0
    R_severe_home_coeficeint=0
    R_R_fatal_coeficeint=0
    C_coeficeint=1
    D_coeficeint=0
    
    coeficeint=[S_coeficeint,E_coeficeint,I_coeficeint,R_mild_coeficeint,R_severe_coeficeint,R_severe_home_coeficeint,R_severe_home_coeficeint,R_R_fatal_coeficeint,C_coeficeint,D_coeficeint]
    
    objective=0
    for i in range(8):
        objective+=-coeficeint[i]*np.sum(states_int_array[i]/1e8)
    return(objective)

In [6]:
calculate_opt([np.array([100,151]), np.array([50,50]), np.array([1,0.5])])

-18.39555325